## 5. Algorithmic Question 

Federico studies in a demanding university where he has to take a certain number $N$ of exams to graduate,  but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system. 

He was given an initial personal score of $S$ when he enrolled, which changes every time he takes an exam: now comes the crazy part.
He soon discovered that every of the $N$ exams he has to take is assigned a mark $p$. Once he has chosen an exam, his score becomes equal to the mark $p$, and at the same time, the scoring system changes:
+ If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity $S - p$.
+ If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity $p - S$.

So, for example, consider $S=8$ as the initial personal score. Federico must decide which exam he wants to take, being $[5,7,1]$ the marks list. If he takes the first one, being $5 < 8$ and $8 - 5 = 3$, the remaining list now becomes $[10,4]$, and his score is updated as $S = 5$.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line,  and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get. 

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.

b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university! 

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

Here are some input/output examples (the first value is the initial personal score, and the second line contains the list of marks): 

__Input 1__
```
8
5 7 1 
```

__Output 1__
```
11
```

__Input 2__
```
25
18 24 21 32 27
```

__Output 2__
```
44
```

__Input 3__
```
30
13 27 41 59 28 33 39 19 52 48 55 79
```

__Output 3__
```
205
```

### 5.1. Recursive approach 

Federico requires an algorithm that allows students to achieve their optimal personal score through a *recursive* approach. Our initial strategy involves utilizing the DFS (Depth-First Search) algorithm. The methodology is as follows:

1. Implement a `for` loop where we process each element in the array. This involves:
   - Setting the current element as the new score.
   - Calculating the total score.
   - Creating a modified array of exams, excluding the current element, and updating this array while retaining the original array.

2. Determine the maximum score by comparing the current `max_score` with the result of a recursive call, which takes the new score and the updated exams array as parameters.

3. Continue this process until all possible iterations through the array are completed.


In [67]:
def exams_v0(score, exams):
    if len(exams) == 1:
        return exams[0]

    max_score = 0
    for i in range(len(exams)):
        new_score = exams[i]
        new_quantity = score - new_score
        new_exams = [x + new_quantity for idx, x in enumerate(exams) if idx != i]
        max_score = max(max_score, exams_v0(new_score, new_exams))

    return max_score


print(exams_v0(8, [5, 7, 1]))  # Expected output: 11
print(exams_v0(25, [18, 24, 21, 32, 27]))  # Expected output: 44
print(exams_v0(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]))  # Expected output: 205

11
44
205


However, this approach becomes significantly slow when dealing with a large number of examples (as seen in example 3). A different strategy is required. We propose implementing the DFS algorithm with the addition of memoization to track previous states. This method essentially constitutes Dynamic Programming. By employing this approach, we expect:

- **Improved Time Complexity:** The overall processing time should be significantly reduced.
- **Space Trade-Off:** Additional memory will be required to store all the states, which is a trade-off for the enhanced time efficiency.


In [1]:
def exams_v1(score, exams, memo={}):
    if len(exams) == 1:
        return exams[0]
    if (score, tuple(exams)) in memo:
        return memo[(score, tuple(exams))]

    max_score = 0
    for i in range(len(exams)):
        new_score = exams[i]
        new_quantity = score - new_score
        new_exams = [x + new_quantity for idx, x in enumerate(exams) if idx != i]
        max_score = max(max_score, exams_v1(new_score, new_exams, memo))

    memo[(score, tuple(exams))] = max_score
    return max_score


print(exams_v1(8, [5, 7, 1]))  # Expected output: 11
print(exams_v1(25, [18, 24, 21, 32, 27]))  # Expected output: 44
print(exams_v1(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]))  # Expected output: 205

11
44
205


When comparing the running times of the two approaches, the benefits of memoization become evident:

- **Without Memoization:** The running time is approximately 9 minutes and 21 seconds.
- **With Memoization:** The running time is significantly reduced to just 0.6 seconds.

Given these results, this approach is markedly more efficient for our current situation. Particularly in systems handling a large number of students, if the space constraint is not an issue, state memoization can greatly expedite the computation of maximum scores. This is especially true in scenarios where similar problems have already been encountered, allowing the system to leverage previous computations to speed up the process.


### 5.2. Big-O notation for the recursive algorithm 

The `exams_v1` function is a recursive algorithm with memoization, so analyzing its time complexity can be difficult.

The time complexity, without memoization, would be exponential in the size of the `exams` list, approximately `O(n!)`, where `n` is the length of the `exams` list. This is due to exploring all possible combinations of exams in recursive style. 

With memoization, the time complexity is significantly reduced in average case. The complexity largely depends on the number of unique states, which are determined by the combination of the current score and the remaining exams. However, still in the worst case scenario the time complexity remains `O(n!)`, with also having much worse space complexity. 


### 5.3. New approach!

A different approach is necessary. After exploring various strategies, the most effective one seems to involve sorting the `exams` array and then dividing it into two arrays: `easier_exams` and `harder_exams`. For an uneven number of exams, the `harder_exams` array will contain one additional element. The process unfolds as follows:

1. **Initial Selection:**
   - Select the easiest exam in `harder_exams` if the length of `harder_exams` is larger than the total in `easier_exams`.
   - Otherwise, opt for the easiest exam from `easier_exams`.

2. **Score Assignment:**
   - This selected exam becomes the new score.
   - Remove the chosen exam from its array.

3. **Calculation and Update:**
   - Calculate the necessary quantity.
   - Update both the `easier_exams` and `harder_exams` arrays accordingly.

4. **Iteration:**
   - Update the current score to the value of the `new_score`.
   - Continue this process until all exams are addressed.

The idea behind this strategy is to tackle the hardest exam at the end, thereby benefiting from the increased value accumulated from the last easy exam.

In [1]:
def exams_v2(score, exams):
    if len(exams) == 1:
        return exams[0]

    sorted_exams = sorted(exams)
    easier_exams = sorted_exams[: (len(exams) // 2)]
    harder_exams = sorted_exams[len(exams) // 2 : len(exams)]
    while len(harder_exams) > 0:
        if len(harder_exams) == len(easier_exams):
            new_score = easier_exams[0]
            del easier_exams[0]
        else:
            new_score = harder_exams[0]
            del harder_exams[0]

        new_quantity = score - new_score
        harder_exams = [x + new_quantity for x in harder_exams]
        easier_exams = [x + new_quantity for x in easier_exams]
        score = new_score

    return score


print(exams_v2(8, [5, 7, 1]))  # Expected output: 11
print(exams_v2(25, [18, 24, 21, 32, 27]))  # Expected output: 44
print(exams_v2(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])) # Expected output: 205

11
44
205


After checking the results, they match what we got from the earlier algorithms, suggesting our new implementation is correct.

#### Time complexity

The time complexity of the `exams_v2` function can be broken down as follows:

1. **Sorting the Exams Array:**
   - The `sorted()` function is used, which typically has a time complexity of `O(n log n)` where `n` is the number of elements in the `exams` array.

2. **Splitting the Array:**
   - Dividing the array into `easier_exams` and `harder_exams` is a constant time operation, `O(1)`, involving basic arithmetic and slicing.

3. **While Loop:**
   - The loop iterates over `harder_exams`, potentially up to `n/2` times in the worst case.
   - Within each iteration:
     - Removing the first element (`del`) is `O(n)` as it requires shifting all elements.
     - Updating `harder_exams` and `easier_exams` is `O(n)` for each list, totaling `O(n)` per iteration.

Combining these complexities:

- The initial sorting is `O(n log n)`.
- The while loop, with its operations, contributes `O(n^2/2)`.

`O(n^2/2)` takes priority over `O(n log n)`, therfore, the overall time complexity is approximately `O(n^2)`.

When compared to our previous implementations, this approach is significantly faster and does not require the additional space that was needed for memoization. Thus, Federico is likely to be more satisfied with this solution.

### 5.4. Third approach (ChatGPT)

For our final approach, we plan to consult ChatGPT for a third, optimized implementation. Our objective is to:

1. Provide ChatGPT with the latest version of our code.
2. Clearly explain the task requirements.
3. Analyze the time complexity of the suggested optimized solution.

This will help us determine if there is an improvement in time complexity compared to our previous methods.


Prompt that was used is following: 

"I need a optimized solution for this task: `(explanation of the task)`. The code that was already done is next one:
```python
def exams_v2(score, exams):
    if len(exams) == 1:
        return exams[0]

    sorted_exams = sorted(exams)
    easier_exams = sorted_exams[: (len(exams) // 2)]
    harder_exams = sorted_exams[len(exams) // 2 : len(exams)]
    while len(harder_exams) > 0:
        if len(harder_exams) == len(easier_exams):
            new_score = easier_exams[0]
            del easier_exams[0]
        else:
            new_score = harder_exams[0]
            del harder_exams[0]

        new_quantity = score - new_score
        harder_exams = [x + new_quantity for x in harder_exams]
        easier_exams = [x + new_quantity for x in easier_exams]
        score = new_score

    return score


print(exams_v2(8, [5, 7, 1]))  # Expected output: 11
print(exams_v2(25, [18, 24, 21, 32, 27]))  # Expected output: 44
print(exams_v2(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])) # Expected output: 205
```
One of the problems is O(n^2) complexity inside the while loop, so different approach is needed."



After a few prompts and explaining the problem in more detail, the code that was produced was next one:

In [5]:
from collections import deque

def exams_v3(score, exams):
    if len(exams) == 1:
        return exams[0]

    sorted_exams = sorted(exams)
    mid_index = len(exams) // 2
    easier_exams = deque(sorted_exams[:mid_index])
    harder_exams = deque(sorted_exams[mid_index:])

    cumulative_adjustment = 0

    while harder_exams or easier_exams:
        if len(harder_exams) == len(easier_exams):
            current_exam = easier_exams.popleft()
        else:
            current_exam = harder_exams.popleft()

        current_exam += cumulative_adjustment
        new_score = score - current_exam
        cumulative_adjustment += new_score
        score = current_exam

    return score

print(exams_v3(8, [5, 7, 1]))  # Expected output: 11
print(exams_v3(25, [18, 24, 21, 32, 27]))  # Expected output: 44
print(exams_v3(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])) # Expected output: 205


11
44
205


The primary difference between these two codes lies in the use of `deque` instead of a regular list. With `deque`, we can utilize `popleft()`, which is an `O(1)` operation, in contrast to the `O(n)` operation when using regular lists. This approach also avoids recalculating all remaining exam scores in each iteration. Instead, it updates a `cumulative_adjustment` variable, which is now also an `O(1)` operation. As a result, the overall time complexity is reduced to `O(nlog(n))`, making it a much more optimized solution compared to the `exams_v2` approach.

Additionally, with the use of ChatGPT version 4, prompt was asked to compare their execution times. Here are the results:
- `exams_v2`: approximately 0.060 seconds
- `exams_v3`: approximately 0.00084 seconds
